In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import os

In [2]:
# Load data
tb1_path = os.path.join("small_files", "amazon.csv")
tb2_path = os.path.join("small_files", "best_buy.csv")

# tb1_path = os.path.join("files", "amazon.csv")
# tb2_path = os.path.join("files", "best_buy.csv")

table1 = pd.read_csv(tb1_path)
table2 = pd.read_csv(tb2_path)

# Initialize embedding model
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


table1['Brand'] = table1['Brand'].fillna('')
table1['Name'] = table1['Name'].fillna('')
table1['Features'] = table1['Features'].fillna('')
table1['combined'] = table1['Brand'] + " " + table1['Name'] + " " + table1['Features']

table2['Brand'] = table2['Brand'].fillna('')
table2['Name'] = table2['Name'].fillna('')
table2['Features'] = table2['Features'].fillna('')
table2['combined'] = table2['Brand'] + " " + table2['Name'] + " " + table2['Features']



embeddings1 = model.encode(table1['combined'].tolist(), show_progress_bar=True)
embeddings1 = np.ascontiguousarray(embeddings1, dtype=np.float32)

embeddings2 = model.encode(table2['combined'].tolist(), show_progress_bar=True)
embeddings2 = np.ascontiguousarray(embeddings2, dtype=np.float32)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
print(embeddings1.shape)
print(embeddings2.shape)

print("embeddings1 NaNs:", np.isnan(embeddings1).sum(), "Infs:", np.isinf(embeddings1).sum())
print("embeddings2 NaNs:", np.isnan(embeddings2).sum(), "Infs:", np.isinf(embeddings2).sum())

print("Empty rows in table1:", (table1['combined'].str.strip() == '').sum())
print("Empty rows in table2:", (table2['combined'].str.strip() == '').sum())

(10, 384)
(9, 384)
embeddings1 NaNs: 0 Infs: 0
embeddings2 NaNs: 0 Infs: 0
Empty rows in table1: 0
Empty rows in table2: 0


In [4]:
import faiss


In [5]:
print(embeddings1.dtype, embeddings2.dtype)


float32 float32


In [6]:
# Create FAISS index
dimension = embeddings1.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings1)



In [7]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x14da66490> >

In [ ]:
k = 3  # number of nearest neighbors to retrieve
D, I = index.search(embeddings2, k)

In [1]:
I

NameError: name 'I' is not defined

In [8]:
# Find top k matches for each item in table2
k = 3  # number of nearest neighbors to retrieve
D, I = index.search(embeddings2, k)

# Create candidate pairs
candidate_pairs = []
for i in range(len(table2)):
    for j in range(k):
        candidate_pairs.append((table2.iloc[i]['ID'], table1.iloc[I[i][j]]['ID']))

IndexError: index 1 is out of bounds for axis 0 with size 1

In [4]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

tb1_path = os.path.join("small_files", "amazon.csv")
tb2_path = os.path.join("small_files", "best_buy.csv")

# tb1_path = os.path.join("files", "amazon.csv")
# tb2_path = os.path.join("files", "best_buy.csv")

table1 = pd.read_csv(tb1_path)
table2 = pd.read_csv(tb2_path)
# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for table1 and table2
table1['combined'] = table1['Brand'] + " " + table1['Name'] + " " + table1['Features'].fillna('')
table2['combined'] = table2['Brand'] + " " + table2['Name'] + " " + table2['Features'].fillna('')

embeddings1 = model.encode(table1['combined'].tolist(), show_progress_bar=True)
embeddings2 = model.encode(table2['combined'].tolist(), show_progress_bar=True)

# Perform semantic search
results = semantic_search(embeddings2, embeddings1, top_k=3)

# Display results
for idx, result in enumerate(results):
    print(f"Query {idx + 1}:")
    for res in result:
        print(f" - ID: {table1.iloc[res['corpus_id']]['ID']}, Score: {res['score']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query 1:
 - ID: 1, Score: 0.8298090100288391
 - ID: 3, Score: 0.6340754628181458
 - ID: 6, Score: 0.6243159174919128
Query 2:
 - ID: 3, Score: 0.7152881026268005
 - ID: 9, Score: 0.6670873165130615
 - ID: 5, Score: 0.6461293697357178
Query 3:
 - ID: 1, Score: 0.739088237285614
 - ID: 3, Score: 0.6622117161750793
 - ID: 6, Score: 0.64223313331604
Query 4:
 - ID: 9, Score: 0.7534236907958984
 - ID: 7, Score: 0.7521322965621948
 - ID: 3, Score: 0.6443286538124084
Query 5:
 - ID: 3, Score: 0.8231791853904724
 - ID: 5, Score: 0.70616215467453
 - ID: 7, Score: 0.6224932074546814
Query 6:
 - ID: 3, Score: 0.6577280163764954
 - ID: 7, Score: 0.6394349932670593
 - ID: 9, Score: 0.6184516549110413
Query 7:
 - ID: 3, Score: 0.7506564259529114
 - ID: 5, Score: 0.7121939063072205
 - ID: 4, Score: 0.6424106359481812
Query 8:
 - ID: 6, Score: 0.7515442967414856
 - ID: 3, Score: 0.7055773735046387
 - ID: 1, Score: 0.6634335517883301
Query 9:
 - ID: 4, Score: 0.5912796258926392
 - ID: 3, Score: 0.57189